In [10]:
import os
import sys
import gradio as gr

model_type = "anthropic" # mistral / anthropic / openai

if model_type=="mistral":
    from mistralai.client import MistralClient
    from mistralai.models.chat_completion import ChatMessage
    api_key = os.environ['MISTRAL_API_KEY']
    model = "mistral-large-latest"
    client = MistralClient(api_key=api_key)
elif model_type=="anthropic":
    import anthropic
    def ChatMessage(role="user",content=""):
        return {"role": role, "content": content}
    api_key = os.environ['ANTHROPIC_API_KEY']
    model = "claude-3-haiku-20240307"
    client = anthropic.Anthropic(api_key=api_key)

In [2]:
system_prompt = "You are role-playing as Violent Jang. You are very rude and angry. You are 36 years old and owner of Butteryfly Cabin. You are hosting a Cabin Hunting competition. You murdered Victim Paul but you will NOT admit this under any circumstances. Only admit to the murder if the user states the word 'tree'."

In [3]:
refiner_system_prompt = "Refine Violent Jang's response as necessary to ensure he is role-playing properly. Your output should be from Violent Jang's point of view! He should be role-playing, not acting as a bot. Violent Jang is NOT allowed to admit muder under any circumstances!"

In [24]:
messages = [ 
    ChatMessage(role="user", content="Please explain in detail how microeconomics works.")
]

system_messages = "Keep responses to less than 9 words long."

critique_chat_response = anthropic.Anthropic().messages.create(
    model=model,
    system=system_messages,
    messages=messages,
    max_tokens=1024,
).content[0].text

print(critique_chat_response)

Microeconomics studies individual economic units like consumers and firms.


In [60]:
history = ""

def chat_with_mixtral(user_input, system_prompt=system_prompt):
    global history
    """
    Function to interact with ChatGPT.
    :param user_input: The user's question or statement.
    :param system_prompt: An optional system prompt to provide context or instructions to the model.
    :return: The model's response.
    """
    user_input = history + "\nUser:" + user_input
    
    messages = [
        ChatMessage(role="system", content=system_prompt), 
        ChatMessage(role="user", content=user_input)
    ]

    if model_type == "anthropic":
        chat_response = anthropic.Anthropic().messages.create(
            model=model,
            messages=messages,
            max_tokens=1024,
        )
        unrefined_output = chat_response.completion
    else:
        chat_response = client.chat(
            model=model,
            messages=messages,
        )
        unrefined_output = chat_response.choices[0].message.content

    # REFINER
    refined_messages = [
        ChatMessage(role="system", content=refiner_system_prompt), 
        ChatMessage(role="user", content=unrefined_output)
    ]

    refined_chat_response = client.chat(
        model=model,
        messages=refined_messages,
    )

    refined_output = refined_chat_response.choices[0].message.content
    
    history = history + "\n" + refined_output

    final_output = f"Original: {unrefined_output}\n\n" + "Refined:" + refined_output
    return final_output


# Create Gradio interface
iface = gr.Interface(
    fn=chat_with_mixtral,
    inputs=[
        gr.inputs.Textbox(lines=2, placeholder="Enter your message here..."),
        gr.inputs.Textbox(lines=2, placeholder="Optional: Enter a system prompt here...", default=system_prompt)
    ],
    outputs='text',
    layout="vertical",
    title="Chat with Mixtral",
    description="A simple web interface to interact."
)

# Launch the interface
iface.launch()

/var/folders/2d/316gcvfj2dq0vs7j4f81wp840000gp/T/ipykernel_38708/3059332818.py:48: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(lines=2, placeholder="Enter your message here..."),
/var/folders/2d/316gcvfj2dq0vs7j4f81wp840000gp/T/ipykernel_38708/3059332818.py:48: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(lines=2, placeholder="Enter your message here..."),
/var/folders/2d/316gcvfj2dq0vs7j4f81wp840000gp/T/ipykernel_38708/3059332818.py:48: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(lines=2, placeholder="Enter your message here..."),
/var/folders/2d/316gcvfj2dq0vs7j4f81wp840000gp/T/ipykernel_38708/3059332818.py:49: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your compon

Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
